# Import Required Libraries
Import the necessary libraries, including MetaTrader 5 and pandas.

In [ ]:
# Import the necessary libraries
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime

# Initialize MetaTrader 5 Connection
Initialize the MetaTrader 5 connection and handle any initialization errors.

In [ ]:
# Initialize MetaTrader 5 connection
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()
else:
    print("connected succesfully")

# Set Symbol and Timeframe
Set the symbol for which to stream live ticks.

In [ ]:
# Set Symbol and Timeframe

# Set the symbol for which to stream live ticks
symbol = "EURGBP"
update_ticks_seconds = 0.1

# Ensure the symbol is available in the market watch
if not mt5.symbol_select(symbol, True):
    print(f"Failed to select symbol {symbol}")
    mt5.shutdown()

# Process and Display Ticks
Process the retrieved tick data and display it in a readable format, such as a pandas DataFrame.

In [ ]:
import pandas as pd
import time
import MetaTrader5 as mt5
import threading
import matplotlib.pyplot as plt

# Add at the start of your code
stop_signal = threading.Event()

# Initialize empty DataFrame
tick_history = pd.DataFrame()

# Modify the stream_live_ticks function to use the new processing function
def stream_live_ticks(symbol):
    global tick_history
    
    while not stop_signal.is_set():  # Check if stop was requested
        # Request tick data
        ticks = mt5.symbol_info_tick(symbol)
        
        if ticks is None:
            print("No tick data retrieved")
        else:
            # Create DataFrame for current tick
            current_tick = pd.DataFrame([ticks._asdict()])
            current_tick['time'] = pd.to_datetime(current_tick['time'], unit='s')
            current_tick = current_tick.drop(columns=['last','time_msc', 'flags', 'volume', 'volume_real'])

            # Append to history
            tick_history = pd.concat([tick_history, current_tick], ignore_index=True)
            
            # Display latest data
            print(tick_history.tail(1))
    
        time.sleep(update_ticks_seconds)  # Add small delay to prevent CPU hogging

# Run the stream_live_ticks function in a separate thread
thread = threading.Thread(target=stream_live_ticks, args=(symbol,))
thread.start()

# Function to stop the streaming
def stop_streaming():
    stop_signal.set() #this will stop the while loop in the thread
    thread.join()  # Wait for thread to finish
    print("Streaming stopped")


                   time      bid      ask
574 2024-11-14 02:48:34  0.83148  0.83165
                   time      bid      ask
575 2024-11-14 02:48:34  0.83148  0.83165
                   time      bid      ask
576 2024-11-14 02:48:34  0.83148  0.83165
                   time      bid      ask
577 2024-11-14 02:48:34  0.83148  0.83165
                   time      bid      ask
578 2024-11-14 02:48:34  0.83148  0.83165
                   time      bid      ask
579 2024-11-14 02:48:34  0.83148  0.83165
                   time      bid      ask
580 2024-11-14 02:48:34  0.83148  0.83165
                   time      bid      ask
581 2024-11-14 02:48:34  0.83148  0.83165
                   time      bid      ask
582 2024-11-14 02:48:34  0.83148  0.83165
                   time      bid      ask
583 2024-11-14 02:48:34  0.83148  0.83165
                   time      bid      ask
584 2024-11-14 02:48:34  0.83148  0.83165
                   time      bid      ask
585 2024-11-14 02:48:34  0.83148  

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib
matplotlib.use('TkAgg')  # Use TkAgg backend for interactive plotting

# Assuming your DataFrame is called 'df' and has columns 'timestamp' and 'value'
fig, ax = plt.subplots(figsize=(10, 6))
def update(frame):
    ax.clear()
    # Plot only the last n rows of the dataframe
    ax.plot(tick_history['time'].tail(600), tick_history['bid'].tail(600), label='Bid', color='red')
    ax.plot(tick_history['time'].tail(600), tick_history['ask'].tail(600), label='Ask', color='blue')
    ax.legend()
    ax.set_title('EUR/GBP Data Plot')
    ax.grid(True)
    # Rotate x-axis labels for better readability
    plt.xticks(rotation=45)
    fig.tight_layout()

# Create animation that updates measured in milliseconds (1000 ms = 1 second)
ani = FuncAnimation(fig, update, interval=update_ticks_seconds*1000)

# Show the plot
plt.show()


# Explanation of FuncAnimation Live Chart

The `FuncAnimation` class from the `matplotlib.animation` module is used to create animations by repeatedly calling a function (`update` in this case) at specified intervals. Here's a detailed breakdown of how it works in the context of the live chart:

1. **Importing Required Libraries**: The necessary libraries for plotting and animation are imported, including `matplotlib.pyplot` and `FuncAnimation`.

2. **Setting Up the Plot**: A figure and axis are created using `plt.subplots()`. This is where the live data will be plotted.

3. **Defining the Update Function**: The `update` function is defined to clear the axis and plot the latest data from the `tick_history` DataFrame. The x-axis labels are rotated for better readability, and the layout is adjusted to fit the plot elements properly.

4. **Creating the Animation**: The `FuncAnimation` class is used to create the animation. It takes the figure, update function, and interval (in milliseconds) as arguments. In this case, the plot is updated every 5000 milliseconds (5 seconds).

5. **Displaying the Plot**: The `plt.show()` function is called to display the plot. This will open a window showing the live chart, which updates at the specified interval.

### Example Usage

To create and display a live chart using `FuncAnimation`, you would run the provided code. The chart will update every 5 seconds with the latest tick data, providing a real-time visualization of the bid prices.

In [ ]:
# To stop the stream, call:
stop_streaming()

In [ ]:
print(tick_history)

# Explanation of the stop_streaming Function

The `stop_streaming` function is designed to safely stop the live tick data streaming process. Here's a detailed breakdown of what each part of the function does:

1. **`stop_signal.set()`**: This line sets the `stop_signal` event. The `stop_signal` is a threading event that is used to signal the streaming thread to stop. When `stop_signal.set()` is called, the event is set to true, which the streaming thread checks in its loop to determine if it should exit.

2. **`thread.join()`**: This line waits for the streaming thread to finish its execution. The `join` method blocks the calling thread (in this case, the main thread) until the thread whose `join` method is called is terminated. This ensures that the streaming thread has completely stopped before proceeding.

3. **`print("Streaming stopped")`**: This line simply prints a message to the console indicating that the streaming has been stopped. This is useful for debugging and confirming that the function has executed successfully.

### Example Usage

To stop the live tick data streaming, you would call the `stop_streaming` function. This will set the stop signal, wait for the streaming thread to finish, and then print the confirmation message.
